In [1]:
from wrappers import Arctic, GPT

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool


from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
# members = ["SQL-Generator", "Context-Retriever"]
members = ["SQL-Generator"]

system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = GPT(model="gpt-4-1106-preview")

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [3]:
def create_agent(llm: GPT, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [9]:
import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str

@tool
def hello_world(string: str) -> str:
    """This is a simple tool that just returns a string"""
    return "hello world"


research_agent = create_agent(llm, [hello_world], "You are good at generating SQL commands.")
research_node = functools.partial(agent_node, agent=research_agent, name="SQL-Generator")

# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION

workflow = StateGraph(AgentState)
workflow.add_node("SQL-Generator", research_node)
workflow.add_node("supervisor", supervisor_chain)

In [10]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.set_entry_point("supervisor")

graph = workflow.compile()

In [11]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Create a SQL database that holds information useful for project managers in software development.")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'SQL-Generator'}}
----
{'SQL-Generator': {'messages': [HumanMessage(content="Creating a SQL database involves multiple steps: defining the database schema, creating the database, and then creating tables within that database. Here's a conceptual outline of how a project manager in software development might want their database structured, along with some SQL commands to create such a structure.\n\nFirst, we need to decide what kind of information is useful for project managers. Common entities in software development projects include:\n\n1. Projects: Information about individual projects.\n2. Team Members: Information about people working on the projects.\n3. Tasks: Details about tasks within each project.\n4. Sprints: Information for agile sprint planning.\n5. Issues: Tracking bugs or other issues.\n6. Time Tracking: Information on time spent on tasks or projects.\n\nLet's define a simple schema with these entities:\n\n```sql\n-- Create a database named 'Projec